In [1]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

# using riotwatcher wrapper & setting some required variables

lol_watcher = LolWatcher('RGAPI-XXXXXXXXXXXXXXXXXXXXXXXXXXXXXX')
my_region = 'na1'

In [2]:
# getting few more varibales needed from riot api

me = lol_watcher.summoner.by_name(my_region, 'ìrelia')

summonerId = me['id']
accountId = me['accountId']

In [3]:
# getting a matchlist

matchlist = lol_watcher.match.matchlist_by_account(my_region, accountId)
correctedMatchlist = matchlist['matches']

In [4]:
# getting a list of gameId from past 100 games
listGameId = [match['gameId'] for match in correctedMatchlist]

# manually removing my personal ranked games that are remakes
recentGames = listGameId[0:4]+ listGameId[5:26]+ listGameId[27:100]

In [6]:
# created dataframe from stats given by league api
RecentGameInfo = [lol_watcher.match.by_id(my_region, game) for game in recentGames]
participantInfo = []
summonerIdInfo = []
for game in RecentGameInfo:
    if game['gameMode'] == 'CLASSIC':        # only iterate through ranked games
        for row in game['participants']:
            leagueData = {}
            leagueData['teamId'] = row['teamId']
            leagueData['participantId'] = row['participantId']
            leagueData['champion'] = row['championId']
            leagueData['role'] = row['timeline']['role']
            leagueData['lane'] = row['timeline']['lane']        
            leagueData['spell1'] = row['spell1Id']
            leagueData['spell2'] = row['spell2Id']
            leagueData['item1'] = row['stats']['item0']
            leagueData['item2'] = row['stats']['item1']
            leagueData['item3'] = row['stats']['item2']
            leagueData['item4'] = row['stats']['item3']
            leagueData['item5'] = row['stats']['item4']
            leagueData['item6'] = row['stats']['item5']
            leagueData['item7'] = row['stats']['item6']
            leagueData['win'] = row['stats']['win']
            leagueData['kills'] = row['stats']['kills']
            leagueData['deaths'] = row['stats']['deaths']
            leagueData['assists'] = row['stats']['assists']
            leagueData['largestKillingSpree'] = row['stats']['largestKillingSpree']
            leagueData['killingSprees'] = row['stats']['killingSprees']
            leagueData['largestMultiKill'] = row['stats']['largestMultiKill']
            leagueData['doubleKills'] = row['stats']['doubleKills']
            leagueData['tripleKills'] = row['stats']['tripleKills']
            leagueData['quadraKills'] = row['stats']['quadraKills']
            leagueData['pentaKills'] = row['stats']['pentaKills']
            leagueData['longestTimeSpentLiving'] = row['stats']['longestTimeSpentLiving']
            leagueData['totalDamageDealt'] = row['stats']['totalDamageDealt']
            leagueData['magicDamageDealt'] = row['stats']['magicDamageDealt']
            leagueData['physicalDamageDealt'] = row['stats']['physicalDamageDealt']
            leagueData['trueDamageDealt'] = row['stats']['trueDamageDealt']
            leagueData['largestCriticalStrike'] = row['stats']['largestCriticalStrike']
            leagueData['totalDamageDealtToChampions'] = row['stats']['totalDamageDealtToChampions']
            leagueData['magicDamageDealtToChampions'] = row['stats']['magicDamageDealtToChampions']
            leagueData['physicalDamageDealtToChampions'] = row['stats']['physicalDamageDealtToChampions']
            leagueData['trueDamageDealtToChampions'] = row['stats']['trueDamageDealtToChampions']
            leagueData['totalDamageTaken'] = row['stats']['totalDamageTaken']
            leagueData['magicalDamageTaken'] = row['stats']['magicalDamageTaken']
            leagueData['physicalDamageTaken'] = row['stats']['physicalDamageTaken']
            leagueData['trueDamageTaken'] = row['stats']['trueDamageTaken']
            leagueData['goldEarned'] = row['stats']['goldEarned']
            leagueData['goldSpent'] = row['stats']['goldSpent']
            leagueData['turretKills'] = row['stats']['turretKills']
            leagueData['inhibitorKills'] = row['stats']['inhibitorKills']
            leagueData['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
            leagueData['neutralMinionsKilled'] = row['stats']['neutralMinionsKilled']
            leagueData['neutralMinionsKilledTeamJungle'] = row['stats']['neutralMinionsKilledTeamJungle']
            leagueData['neutralMinionsKilledEnemyJungle'] = row['stats']['neutralMinionsKilledEnemyJungle']
            leagueData['champLevel'] = row['stats']['champLevel']
            leagueData['visionWardsBoughtInGame'] = row['stats']['visionWardsBoughtInGame']
            leagueData['sightWardsBoughtInGame'] = row['stats']['sightWardsBoughtInGame']
            leagueData['visionScore'] = row['stats']['visionScore']
            leagueData['wardsPlaced'] = row['stats']['wardsPlaced']
            leagueData['wardsKilled'] = row['stats']['wardsKilled']
            leagueData['firstBloodKill'] = row['stats']['firstBloodKill']
            leagueData['firstBloodAssist'] = row['stats']['firstBloodAssist']
            leagueData['firstTowerKill'] = row['stats']['firstTowerKill']
            leagueData['firstTowerAssist'] = row['stats']['firstTowerAssist']
            leagueData['primaryKeystone'] = row['stats']['perk0']
            leagueData['creepsPerMinDeltas'] = row['timeline']['creepsPerMinDeltas']
            leagueData['xpPerMinDeltas'] = row['timeline']['xpPerMinDeltas']
            leagueData['goldPerMinDeltas'] = row['timeline']['goldPerMinDeltas']
            participantInfo.append(leagueData)
        for row in game['participantIdentities']:
            leagueData2 = {}
            leagueData2['summonerName'] = row['player']['summonerName']
            summonerIdInfo.append(leagueData2)
    else:               # skip aram and normal games
        continue
    
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [7]:
# getting latest version number of each datadragon type
latestChampions = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
latestItems = lol_watcher.data_dragon.versions_for_region(my_region)['n']['item']
latestKeystone = lol_watcher.data_dragon.versions_for_region(my_region)['n']['sticker']
latestSummoner = lol_watcher.data_dragon.versions_for_region(my_region)['n']['summoner']

# getting different information into its own unique list 
championList = lol_watcher.data_dragon.champions(latestChampions, False, 'en_US')
itemsList = lol_watcher.data_dragon.items(latestItems)
itemsList = itemsList['data']
keystoneList = lol_watcher.data_dragon.runes_reforged(latestKeystone)
summonerSpellList = lol_watcher.data_dragon.summoner_spells(latestSummoner)

In [8]:
# creating dictionary of champions with its corresponding number from list created above
champDict = {}
for key in championList['data']:
    row = championList['data'][key]
    champDict[row['key']] = row['id']
for row in participantInfo:
    row['championName'] = champDict[str(row['champion'])]

# updating df to add column w/ champion name based off champion number    
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [9]:
# creating dictionary of summoner spell with its corresponding number from list created above
summonerSpellDict = {}
for key in summonerSpellList['data']:
    row = summonerSpellList['data'][key]
    summonerSpellDict[row['key']] = row['name']
for row in participantInfo:
    row['summonerSpell1'] = summonerSpellDict[str(row['spell1'])]
    row['summonerSpell2'] = summonerSpellDict[str(row['spell2'])]
    
# updating df to add column w/ summoner spell based off summoner spell number       
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [10]:
# creating dictionary corresponding what number is blue/red side
teamDict = {'100': 'Blue', '200': 'Red'}
for row in participantInfo:
    row['team'] = teamDict[str(row['teamId'])]
    
# updating df to add in if the user is in blue or red team
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [11]:
# creating dictionary of keystone with its corresponding number
keystoneDict = {'8112': 'Electrocute',
                '8124': 'Predator',
                '8128': 'Dark Harvest',
                '9923': 'Hail of Blades',
                '8351': 'Glacial Augment',
                '8359': 'Kleptomancy',
                '8360': 'Unsealed Spellbook',
                '8358': 'Prototype: Omnistone',
                '8005': 'Press the Attack',
                '8008': 'Lethal Tempo',
                '8021': 'Fleet Footwork',
                '8010': 'Conqueror',
                '8437': 'Grasp of the Undying',
                '8439': 'Aftershock',
                '8465': 'Guardian',
                '8214': 'Summon Aery',
                '8229': 'Arcane Comet',
                '8230': 'Phase Rush'}
for row in participantInfo:
    row['keystone'] = keystoneDict[str(row['primaryKeystone'])]

# updating df to add in what keystone user used
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [12]:
# creating dictionary of items with its corresponding number from list created above
itemDict = []
for key, value in itemsList.items():
    item_row = {}
    item_row[key] = value['name']
    itemDict.append(item_row)

updatedItemDict = {}
for d in itemDict:
    updatedItemDict.update(d)

updatedItemDict['0'] = 'N/A'   # zero denotes no items in inventory slot

for row in participantInfo:
    row['item-1'] = updatedItemDict[str(row['item1'])]
    row['item-2'] = updatedItemDict[str(row['item2'])]
    row['item-3'] = updatedItemDict[str(row['item3'])]
    row['item-4'] = updatedItemDict[str(row['item4'])]
    row['item-5'] = updatedItemDict[str(row['item5'])]
    row['item-6'] = updatedItemDict[str(row['item6'])]
    row['item-7'] = updatedItemDict[str(row['item7'])]
    
# updating df to add in what items user purchased
dfLeagueStats = pd.DataFrame(participantInfo, summonerIdInfo)

In [13]:
dfLeagueStats.to_csv('master_data_soloq.csv')